In [ ]:
!pip install protobuf==3.20.3 witwidget tensorflow catboost --quiet

import numpy as np
import pandas as pd
import tensorflow as tf
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder
from google.colab import files
import joblib  # For loading trained models


In [ ]:
# Upload dataset
uploaded_dataset = files.upload()

# Identify dataset file
dataset_filename = None

for filename in uploaded_dataset.keys():
    if filename.endswith(".csv"):  # Detects dataset file
        dataset_filename = filename

# Load the dataset if found
if dataset_filename:
    df = pd.read_csv(dataset_filename)
    df_test = df.copy()
    print(f"✅ Dataset '{dataset_filename}' has been successfully loaded.")
else:
    print("⚠️ No dataset file (.csv) found. Please upload a valid dataset.")

In [ ]:
# Upload model
uploaded_model = files.upload()

# Identify model file
model_filename = None

for filename in uploaded_model.keys():
    if filename.endswith(".pkl"):  # Detects model file
        model_filename = filename

# Load the model if found
if model_filename:
    model = joblib.load(model_filename)
    print(f"✅ Model '{model_filename}' has been successfully loaded.")
else:
    print("⚠️ No model file (.pkl) found. Please upload a valid model.")

In [ ]:
# Upload scaler file
uploaded_scaler = files.upload()

# Identify scaler file dynamically
scaler_filename = None

for filename in uploaded_scaler.keys():
    if "scaler" in filename and filename.endswith(".pkl"):
        scaler_filename = filename

# Load scaler if found
scaler = None

if scaler_filename:
    scaler = joblib.load(scaler_filename)
    print(f"✅ Scaler '{scaler_filename}' has been successfully loaded.")
else:
    print("⚠️ No scaler file (.pkl) found. Skipping scaling.")

if scaler:
    # Extract feature columns (excluding "Class")
    feature_columns = [col for col in df.columns if col != "Class"]

    # Apply scaling to features
    X_scaled = scaler.transform(df[feature_columns])
    df_scaled = pd.DataFrame(X_scaled, columns=feature_columns)  # Convert back to DataFrame

    print("✅ Dataset features have been successfully scaled.")
else:
    print("⚠️ Skipping scaling as no scaler file was found.")


In [ ]:
# Upload label encoder file
uploaded_encoder = files.upload()

# Identify label encoder file dynamically
label_encoder_filename = None

for filename in uploaded_encoder.keys():
    if "label_encoder" in filename and filename.endswith(".pkl"):
        label_encoder_filename = filename

# Load label encoder if found
label_encoder = None

if label_encoder_filename:
    label_encoder = joblib.load(label_encoder_filename)
    print(f"✅ Label Encoder '{label_encoder_filename}' has been successfully loaded.")
else:
    print("⚠️ No label encoder file (.pkl) found. Skipping encoding.")

if label_encoder:
    df["Class"] = label_encoder.transform(df["Class"])
    print("✅ Class labels have been successfully encoded.")
else:
    print("⚠️ Skipping encoding as no label encoder file was found.")


In [ ]:
# Ensure feature_columns are correctly defined
feature_columns = [col for col in df.columns if col != "Class"]

# Convert dataset into list format (WIT requires JSON-like structure)
instances = df_scaled.to_dict(orient="records")  # Use scaled dataset

# Define class mapping based on encoded values
class_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
print("Class Mapping:", class_mapping)

# Debugging: Print test predictions using scaled data
test_prediction = model.predict(df_scaled.iloc[:10])  # Use scaled data
print("Raw Model Predictions:", test_prediction)
print(df_scaled.iloc[:10])  # Show scaled feature values
print(df.head(10))  # Show unscaled data for comparison
print(instances[0])  # Show JSON-like instance structure

if "Class" in df_test.columns:
          inputs_df = df_test.drop(columns=["Class"])
instances = df_test.to_dict(orient="records")
print(type(instances))
print(type(instances[0]))
print(instances[0])

In [ ]:
# Define a wrapper function for WIT
class WitModelWrapper:
    def __init__(self, model, class_mapping, scaler):
        self.model = model
        self.class_mapping = class_mapping
        self.scaler = scaler  # Store scaler for prediction

    def predict(self, inputs):

        if isinstance(inputs, dict):  # Ensure inputs is a list of dictionaries
          inputs = [inputs]

        if not all(isinstance(i, dict) for i in inputs):
          raise ValueError("❌ Invalid input format! Expected a list of dictionaries.")

        # Convert inputs to DataFrame
        inputs_df = pd.DataFrame(inputs)
        # Drop the "Class" column if it exists in inputs
        if "Class" in inputs_df.columns:
          inputs_df = inputs_df.drop(columns=["Class"])

        # Ensure correct column order
        inputs_df = inputs_df[feature_columns]

        # Apply scaling before prediction
        inputs_scaled = self.scaler.transform(inputs_df)
        print("Scaled inputs:", inputs_scaled)

        # Get predictions from the model
        preds = self.model.predict(inputs_scaled)
        # Get model predictions (numeric output required)
        preds = self.model.predict_proba(inputs_scaled)  # ✅ Use predict_proba() instead of predict()


        # Convert predictions back to class names
        #categorical_preds = [self.class_mapping[int(pred)] for pred in preds]

        print("Predictions:", preds)  # Debugging output
        return preds.tolist()  # Return categorical predictions

# Wrap the trained model with scaling for WIT predictions
wrapped_model = WitModelWrapper(model, class_mapping, scaler)

# Debugging: Test wrapped model prediction
print("Test Wrapped Model Prediction:", wrapped_model.predict([instances[0]]))


In [ ]:
# Define feature spec for WIT
feature_spec = {feature: tf.io.FixedLenFeature([], tf.float32) for feature in feature_columns}
feature_spec["Class"] = tf.io.FixedLenFeature([], tf.string)  # Ensure "Class" is a string category

# Define class labels
class_labels = list(class_mapping.values())  # Get class names from mapping

config_builder = WitConfigBuilder(instances) \
    .set_custom_predict_fn(wrapped_model.predict) \
    .set_label_vocab(class_labels) \
    .set_target_feature("Class")


# Launch WIT in the notebook
WitWidget(config_builder)